# Análise e Visualização Espacial de Dados Reais em Python com Folium 
Estudo sobre o Fluxo de Voos nas Capitais do Nordeste Brasileiro

Importação das bibliotecas necessárias para manipulação de arquivos CSV e criação e manipulação de mapas.

In [0]:
#Importando as bibliotecas necessárias para analise dos dados
import pandas as pd
import numpy as np
import folium
import json
from folium import plugins
import requests


In [0]:
# install the latest version of folium
!pip install folium==0.8.2

In [0]:
#checking folium version
folium.__version__

In [0]:
# install the latest version of requests
!pip install requests==v2.21.0

2 - Ler os dados sobre aeroportos disponiveis no portal de dados abertos da ANAC (Agência Brasileira de Aviação Civil).

In [0]:
url_aeroportos_publicos = 'https://drive.google.com/uc?authuser=0&id=1xHhzSTRDr6apDqeE5WWElet09HJ-msxj&export=download'

In [0]:
aeroportos_publicos =pd.read_csv(url_aeroportos_publicos,error_bad_lines=False, encoding="latin-1",sep=';')
aeroportos_publicos.info()

3 - Em seguida, é preciso deixar apenas os dados que vamos precisar e descartar colunas não necessárias.

In [0]:
aeroportos_publicos = aeroportos_publicos.drop(aeroportos_publicos.columns[7:], axis=1)
aeroportos_publicos = aeroportos_publicos
aeroportos_publicos.columns = aeroportos_publicos.iloc[1]
aeroportos_publicos= aeroportos_publicos.reindex(aeroportos_publicos.index.drop(0))
aeroportos_publicos= aeroportos_publicos.reindex(aeroportos_publicos.index.drop(1))
aeroportos_publicos = aeroportos_publicos.reset_index(drop=True)
aeroportos_publicos.head()

4 - Agora, é preciso retirar todos os aeroportos que não pertencem ao Nordeste.

In [0]:
todos_estados_sem_nordeste=['AC','AM','AP','PA','RO','RR','TO','GO','MS','MT','ES','MG','RJ','SP','PR','RS','SC','DF']

In [0]:
#dropando estados diferentes do nordeste
def retirar_estados(df):
  for estado in todos_estados_sem_nordeste:
    aeroportos_publicos.drop(aeroportos_publicos[aeroportos_publicos.UF == estado].index,inplace = True)
retirar_estados(aeroportos_publicos)
aeroportos_publicos = aeroportos_publicos.reset_index(drop=True)
aeroportos_publicos.head()

5 - Agora que temos um dataframe apenas com informações de aeroportos da Região Nordeste do Brasil, é preciso transformar as informações de latitude e longitude para o formato que o Folium entende.

In [0]:
aeroportos_publicos.LATITUDE = (aeroportos_publicos.LATITUDE.str.split(r'[°\'"]', expand=True)).iloc[:, :3].astype('float').dot([1, 1./60, 1./3600])*-1

In [0]:
aeroportos_publicos.LATITUDE.head()

In [0]:
aeroportos_publicos.LONGITUDE = aeroportos_publicos.LONGITUDE.str.split(r'[°\'"]', expand=True).iloc[:, :3].astype('float').dot([1, 1./60, 1./3600])*-1

In [0]:
aeroportos_publicos.LONGITUDE.head()

6 - Agora que ja temos um dataframe com todas as informações do Nordeste, precisamos usar Folium para visualizar o mapa da Região Nordeste.

In [0]:
# configure a generical header
headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'User-Agent': 'google-colab',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
}

In [0]:
#Criando mapa com zoom no nordeste brasileiro
mapa_nordeste = folium.Map(location=[-8.8864889,-39.6786708],zoom_start=5.3,
                           tiles='Stamen Terrain')

In [0]:
estado_jsons = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/2/?formato=application/vnd.geo+json&resolucao=2",
                           headers=headers)
estado_json = estado_jsons.json()
estado_json2 = estado_json
folium.GeoJson(estado_json).add_to(mapa_nordeste)

mapa_nordeste

7 - Agora, que já conseguimos ver o mapa do Nordeste, podemos colcoar marcadores nos pontos de cada aeroporto que temos no dataframe.

In [0]:
aeroportos_publicos.head()

In [0]:
tiles = ['openstreetmap','Mapbox Bright','Mapbox Control Room',
        'stamenterrain','stamenterrain','stamentoner',
        'stamenwatercolor','cartodbpositron','cartodbdark_matter']
for tiles_ in tiles: 
  folium.TileLayer(tiles_).add_to(mapa_nordeste)
folium.LayerControl().add_to(mapa_nordeste)

mapa_nordeste

In [0]:
for counter in range(134):
  folium.Marker([aeroportos_publicos.LATITUDE[counter],aeroportos_publicos.LONGITUDE[counter]], 
               popup='<ul><li><h5>'+aeroportos_publicos.NOME[counter]+'</li></h5><ul><li>'
                +aeroportos_publicos['UF'][counter]+'</li><li>'+aeroportos_publicos['MUNICÍPIO ATENDIDO'][counter]+'</ul></li></ul>',
               tooltip=aeroportos_publicos['CÓDIGO OACI'][counter],
               icon=folium.Icon(color='blue',
                    icon_color='black',
                    icon='plane',
                    prefix='fa')).add_to(mapa_nordeste)
  
  mapa_nordeste

In [0]:
#cluster_inutil
data = np.array(
    [
        aeroportos_publicos['LATITUDE'],aeroportos_publicos['LONGITUDE']
    ]
).T

# Create a map
m = folium.Map(
    location=[-8.8864889,-39.6786708],zoom_start=5.3,
    tiles='Stamen Terrain'
)

plugins.MarkerCluster(data).add_to(m)
estado_jsons = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/2/?formato=application/vnd.geo+json&resolucao=2",
                           headers=headers)
estado_json = estado_jsons.json()
folium.GeoJson(estado_json).add_to(m)
m

# Analisando Dados de Emigração e Imigração de Aeroportos do Nordeste Brasileiro (2018)

In [0]:
#Pegar lista aeroportos do nordeste a partir da tabela de aeroportos publicos
estados_ne_2018_dict = {}

siglas_ne_list = list(aeroportos_fed_ne_dict)

for sigla_estado in siglas_ne_list:
  estados_ne_2018_dict[sigla_estado] = aeroportos_publicos[aeroportos_publicos['UF']==sigla_estado]['CÓDIGO OACI']

estados_ne_2018_dict['AL']

In [0]:
aeroportos_rn = aeroportos_publicos[aeroportos_publicos['UF']=='RN']['CÓDIGO OACI']
aeroportos_rn

In [0]:
def clean_df_2018(df_mes):
  df_mes = pd.DataFrame(df_mes, columns=['ICAO Aeródromo Origem',	'ICAO Aeródromo Destino', 'Situação Voo'])
  df_mes.columns = ['aeroporto_origem', 'aeroporto_destino', 'situacao']
  df_mes =  df_mes[(df_mes.situacao == 'REALIZADO')]
  df_mes.reset_index(drop=True)
  return df_mes

In [0]:
#Caminhos para tabelas que contém informações do histórico de voo em 2018
url_fixo_2018 = 'http://www.anac.gov.br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2018/vra'
urls_csv_2018 = ['-01_2018.csv', '-02_2018.csv', '_032018.csv',
                '_04.csv', '_05.csv', '_062018.csv', 
                '_072018.csv', '_082018.csv', '_092018.csv', 
                 '_102018.csv', '_112018.csv', '_122018.csv'] 


In [0]:
frames2018 = []
for url_csv in urls_csv_2018:
  df = clean_df_2018(pd.read_csv( url_fixo_2018 + url_csv ,error_bad_lines=False, encoding="latin-1",sep=';'))
  frames2018.append(df)

In [0]:
result_df_2018 = pd.concat(frames2018, ignore_index=True)
result_df_2018.info()
result_df_2018.loc[1,'aeroporto_origem']

In [0]:
#Alagoas(1), Bahia(2), Ceará(1), Maranhão (3) , Paraiba(2), Pernambuco(2), Piaui(2), RN(1), Sergipe(1)
aeroportos_fed_ne_list = ['SBMO',
                          'SBIL','SBUF',
                          'SBJU',
                          'SBIZ', 'SBSL', 'SNCW',
                          'SBJP', 'SBKG',
                          'SBPL','SBRF',
                          'SBPB','SBTE',
                          'SBNT','SBSG',
                          'SBAR']

aeroportos_fed_ne_dict = {'AL':['SBMO'],
                          'BA':['SBIL','SBUF'],
                          'CE':['SBJU'],
                          'MA':['SBIZ', 'SBSL', 'SNCW'],
                          'PB':['SBJP', 'SBKG'],
                          'PE':['SBPL','SBRF'],
                          'PI':['SBPB','SBTE'],
                          'RN':['SBNT','SBSG'],
                          'SE':['SBAR']}

In [0]:
#Contando quantidade de voos que originam de aeroportos federais do nordeste
voos_origem_estados_ne_2018 = {}

voos_destino_estados_ne_2018 = {}

for estado,aeroportos in aeroportos_fed_ne_dict.items():
  voos_origem_estados_ne_2018[estado] = 0
  voos_destino_estados_ne_2018[estado] = 0
  for aero in aeroportos:
    voos_origem_estados_ne_2018[estado] += result_df_2018[(result_df_2018.aeroporto_origem == aero)].count()[0]
    voos_destino_estados_ne_2018[estado] += result_df_2018[(result_df_2018.aeroporto_destino == aero)].count()[0]

In [0]:
#Transformando o dicionario com dados sobre emigração dos aeroportos do nordeste em dataframe
voos_origem_ne_df_2018 = pd.DataFrame.from_dict({"QTD_VOOS_ORIGEM":voos_origem_estados_ne_2018})
voos_origem_ne_df_2018.index.name = "SIGLA"
voos_origem_ne_df_2018

In [0]:
#Transformando o dicionario com dados sobre imigração dos aeroportos do nordeste em dataframe
voos_destino_ne_df_2018 = pd.DataFrame.from_dict({'QTD_VOOS_DESTINO':voos_destino_estados_ne_2018})
voos_destino_ne_df_2018.index.name = "SIGLA"
voos_destino_ne_df_2018

In [0]:
states = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/regioes/2/estados",
                       headers=headers).json()

states_northeast = [estados["nome"] for estados in states]
states_id = [estados["id"] for estados in states]
state_initials = [estados["sigla"] for estados in states]
data_northeast = pd.DataFrame.from_dict({'ID':states_id,'ESTADO':states_northeast,"SIGLA":state_initials})


In [0]:
final_data_2018 = pd.merge(left = data_northeast,right = voos_origem_ne_df_2018,on="SIGLA",how= "outer")
final_data_2018.set_index('ID', inplace=True)
final_data_2018

In [0]:
final_data_2018_chegada = pd.merge(left = data_northeast,right = voos_destino_ne_df_2018,on="SIGLA",how= "outer")
final_data_2018_chegada.set_index('ID', inplace=True)
final_data_2018_chegada

In [0]:
for state in estado_json2['features']: 
  codarea = state['properties']['codarea']
  state['properties']['sigla'] = str(final_data_2018.loc[int(codarea), "SIGLA"])
  state['properties']['qtd_voos'] = str(final_data_2018.loc[int(codarea), "QTD_VOOS_ORIGEM"])

In [0]:
for state in estado_json2['features']: 
  codarea = state['properties']['codarea']
  state['properties']['sigla'] = str(final_data_2018_chegada.loc[int(codarea), "SIGLA"])
  state['properties']['qtd_voos'] = str(final_data_2018_chegada.loc[int(codarea), "QTD_VOOS_DESTINO"])

In [0]:
#Criando Choropleth 2018 Imigração
z = folium.Map(
    location=[-8.8864889,-39.6786708],zoom_start=5.3,
    tiles='Stamen Terrain'
)

bins = np.linspace(final_data_2018['QTD_VOOS_ORIGEM'].min(),
                   final_data_2018['QTD_VOOS_ORIGEM'].max(),
                   10).tolist()


folium.Choropleth(
    geo_data=estado_json,
    data=final_data_2018,
    name= "ID",
    columns=['SIGLA', 'QTD_VOOS_ORIGEM'],
    key_on='feature.properties.sigla',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='#Voos Saindo do Nordeste 2018',
    bins=bins
).add_to(z)

folium.LayerControl().add_to(z)

z

In [0]:
#Criando Choropleth 2013 Emigração
u = folium.Map(
    location=[-8.8864889,-39.6786708],zoom_start=5.3,
    tiles='Stamen Terrain'
)

bins = np.linspace(final_data_2018_chegada['QTD_VOOS_DESTINO'].min(),
                   final_data_2018_chegada['QTD_VOOS_DESTINO'].max(),
                   10).tolist()


folium.Choropleth(
    geo_data=estado_json2,
    data=final_data_2018_chegada,
    name= "ID",
    columns=['SIGLA', 'QTD_VOOS_DESTINO'],
    key_on='feature.properties.sigla',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='#Voos Chegando no Nordeste',
    bins=bins
).add_to(u)

folium.LayerControl().add_to(u)

u